## Quantum Circuit Simulator

<p>This is a solution for task 3 of the qosf mentorship program.

In [1]:
from quantum_circuit import QuantumCircuit
from utils import *
import numpy as np
from scipy.optimize import minimize

For a 2 qubit system initialize the circuit:

In [2]:
qc = QuantumCircuit(2)

Calculate initial state where all qubits are in state <0|:

In [3]:
initial_state = qc.get_ground_state()
initial_state

array([1., 0., 0., 0.])

Calculate operators for Hadamard gate applied to qubit 0 and a CNOT gate applied to 1 where controlled qubit is at 0:

In [4]:
H = 1./np.sqrt(2) * np.array([[1, 1],[1, -1]])
op1 = qc.op_single_q_gates(H, [0])
op2 = qc.op_cnot([0, 1])

/home/bis/PycharmProjects/qc-sim/quantum_circuit.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if gate_unitary == 'U3':


Calculate final state vector using the measure method of QuantumCricuit class:

In [5]:
final_state = qc.measure(initial_state, [op2, op1])
final_state

array([0.70710678, 0.        , 0.        , 0.70710678])

Finally, for a 1000 shots calculate probability of occurences of each state:

In [6]:
qc.get_counts(final_state)[0]

{'00': 490, '11': 510}

Another example: use H gate on qubit 0. This operation should produce equal probability of finding + and - states.

In [7]:
qc = QuantumCircuit(1)
initial_state = qc.get_ground_state()
op1 = qc.op_single_q_gates(H, [0])
final_state = qc.measure(initial_state, [op1])
print('Final State:', final_state)
print('counts', qc.get_counts(final_state)[0])

Final State: [0.70710678 0.70710678]
counts {'0': 490, '1': 510}


What if we use H gate on qubit 0 followed by another H gate on the same qubit for a 1 qubit system: 

In [8]:
qc = QuantumCircuit(1)
initial_state = qc.get_ground_state()
op1 = qc.op_single_q_gates(H, [0])
op2 = qc.op_single_q_gates(H, [0])
final_state = qc.measure(initial_state, [op2, op1])
print('Final State:', final_state)
print('counts', qc.get_counts(final_state)[0])

Final State: [1. 0.]
counts {'0': 1000}


Use X gate:

In [9]:
X = np.array([[0,1],
            [1,0]])

In [10]:
qc = QuantumCircuit(1)
initial_state = qc.get_ground_state()
op1 = qc.op_single_q_gates(X, [0])
final_state = qc.measure(initial_state, [op1])
final_state

array([0., 1.])

U3 gate:

In [11]:
qc = QuantumCircuit(2)
initial_state = qc.get_ground_state()
op1 = qc.op_single_q_gates('U3', [1], theta = 3.415, phi=1.5708, lambda1=-3.415)
final_state = qc.measure(initial_state, [op1])
final_state

array([-1.36278287e-01+0.j       , -3.63893629e-06+0.9906706j,
        0.00000000e+00+0.j       ,  0.00000000e+00+0.j       ])

Optimize a quantum circuit:

In [12]:
num_shots = 1000

def objective_function(params):
    '''Return cost to optimize a quantum circuit
        Args: 
            params: a list of theta, phi and lambda values
        Returns:
            Mean squared error of counts for each state from total number of shots/2. This is the cost function.
            For a two qubit system this is to be minimized in order to get maximum probabilities at states 
            '00' and '11'.
    '''
    
    theta = params[0]
    phi = params[1]
    lambda1 = params [2]
    qc = QuantumCircuit(2)
    initial_state = qc.get_ground_state()
    op1 = qc.op_single_q_gates('U3', [0], theta=theta, phi = phi, lambda1 = lambda1)
    op2 = qc.op_cnot([0, 1])
    final_state = qc.measure(initial_state, [op2, op1])

    counts, state_count = qc.get_counts(final_state, num_shots=num_shots)
    y_true = [num_shots/2, num_shots/2]  
    mean_squared_error = np.square(np.subtract(y_true, state_count)).mean() 

    return mean_squared_error

In [13]:
params = [3.1415, 1.15708, -3.1415]
objective_function(params)

250000.0

In [14]:
minimum = minimize(objective_function, params, method="Powell", tol=1e-6)
minimum

   direc: array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])
     fun: 25.0
 message: 'Optimization terminated successfully.'
    nfev: 120
     nit: 2
  status: 0
 success: True
       x: array([ 4.71859429, -0.1669482 , -0.52346602])

In [16]:
qc = QuantumCircuit(2)
initial_state = qc.get_ground_state()
op1 = qc.op_single_q_gates('U3', [0], theta=4.71859429, phi = -0.1669482, lambda1 = -0.52346602)
op2 = qc.op_cnot([0, 1])
final_state = qc.measure(initial_state, [op2, op1])
qc.get_counts(final_state)[0]

{'00': 490, '11': 510}